## Installing and Importing the Dependencies

In [1]:
!pip install tensorflow==2.0.0-beta1 
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [0]:
import os
import random
import datetime

import numpy as np
import tensorflow as tf

from tensorflow import keras as krs
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/

## Data, Data Generator and Augmentation

In [0]:
mnist = tf.keras.datasets.mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

## Creating the Model

In [0]:
def create_model(summary=False, name=None):
  model = tf.keras.models.Sequential([
    krs.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    krs.layers.MaxPooling2D(2, 2),
    krs.layers.Flatten(),
    krs.layers.Dense(128, activation='relu'),
    krs.layers.Dense(64, activation='relu'),
    krs.layers.Dense(10, activation='softmax')
  ], name=name)
  if summary==True:
    model.summary()
  return model

In [6]:
model = create_model(summary=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               692352    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 701,578
Trainable params: 701,578
Non-trainable params: 0
__________________________________________________

## Callbacks

In [7]:
log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
chkpoint = './base.model'
train_csv = "training_csv.log"

checkpoint = ModelCheckpoint(
    chkpoint,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)

csvlogger = CSVLogger(
    filename= train_csv,
    separator = ",",
    append = False
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

W0622 21:02:18.039998 140443495036800 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [0]:
callbacks = [tensorboard_callback]

## Tuning, Compiling and Training

In [0]:
# Hyperparameters
total_data = 60000
lr = 0.0001
n_epochs = 50
steps_epoch = total_data / 4
verbosity = 1


# Compiling the Model
model.compile(optimizer=krs.optimizers.Adam(lr=lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
training = model.fit(training_images, 
                     training_labels, 
                     epochs=n_epochs,
                     #steps_per_epoch=steps_epoch,
                     verbose=verbosity,
                     callbacks=callbacks)

W0622 21:02:18.168382 140443495036800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 27s 452us/sample - loss: 0.3707 - accuracy: 0.9010
Epoch 2/50
60000/60000 [==============================] - 27s 445us/sample - loss: 0.1439 - accuracy: 0.9582
Epoch 3/50
60000/60000 [==============================] - 27s 442us/sample - loss: 0.0969 - accuracy: 0.9717
Epoch 4/50
60000/60000 [==============================] - 26s 439us/sample - loss: 0.0720 - accuracy: 0.9789
Epoch 5/50
60000/60000 [==============================] - 26s 437us/sample - loss: 0.0581 - accuracy: 0.9832
Epoch 6/50
60000/60000 [==============================] - 26s 433us/sample - loss: 0.0483 - accuracy: 0.9863
Epoch 7/50
60000/60000 [==============================] - 26s 437us/sample - loss: 0.0416 - accuracy: 0.9879
Epoch 8/50
60000/60000 [==============================] - 26s 440us/sample - loss: 0.0356 - accuracy: 0.9897
Epoch 9/50
60000/60000 [==============================] - 26s 434us/sample - loss: 0.0305 - accuracy: 0.9

## Showing the Results

In [14]:
# Starting Tensorboard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 655), started 0:01:10 ago. (Use '!kill 655' to kill it.)

## Saving and Loading the Results

In [0]:
## Saving and Loading the WEIGHTS
model.save_weights('my_model_weights.h5')
model.load_weights('my_model_weights.h5')

In [0]:
# Saving and Loading the whole model (ARCHITECTURE + WEIGHTS + OPTIMIZER STATE)
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')